<a href="https://colab.research.google.com/github/imabari/covid19-data/blob/master/aichi/aichi_patients_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
from bs4 import BeautifulSoup

In [0]:
from urllib.parse import urljoin

In [0]:
import re

In [0]:
import datetime

In [0]:
url = "https://www.pref.aichi.jp/site/covid19-aichi/kansensya-kensa.html"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko",
}

In [0]:
r = requests.get(url, headers=headers)

r.raise_for_status()

soup = BeautifulSoup(r.content, "html5lib")

In [0]:
tag = soup.find("a", text=re.compile("県内発生事例一覧.+Excelファイル"))

In [0]:
link = urljoin(url, tag.get("href"))

In [0]:
m = re.search("(\d{1,2})月(\d{1,2})日", tag.get_text(strip=True))

In [0]:
dt_now = datetime.datetime.now()

In [0]:
month, day = map(int, m.groups())

In [0]:
dt_update = datetime.datetime(dt_now.year, month, day)

# テスト
# dt_update = datetime.datetime(dt_now.year, month, day) + datetime.timedelta(days=7)

In [237]:
dt_update

datetime.datetime(2020, 4, 22, 0, 0)

In [0]:
import pandas as pd

In [0]:
import json

In [0]:
data = {}

In [0]:
df_kanja = pd.read_excel(link, index_col="No", header=2)

In [242]:
df_kanja

,発表日,年代・性別,国籍,住居地,接触状況,備考
No,,,,,,
1,43856,40代男性,中国,中国武漢市,中国,本県発表1
2,43858,40代男性,中国,中国武漢市,中国,本県発表2
3,43875,60代男性,日本,名古屋市,アメリカ,名古屋市発表1
4,43876,60代女性,日本,名古屋市,No.3の妻,名古屋市発表2
5,43877,60代男性,日本,尾張地方,No.4と接触,本県発表3
...,...,...,...,...,...,...
445,43943,90代男性,0,名古屋市,No.410と接触,名古屋市発表252
446,43943,60代女性,0,名古屋市,No.445と接触,名古屋市発表253
447,43943,20代女性,0,名古屋市,No.417と接触,名古屋市発表254


In [0]:
df_kanja["発表日"] = df_kanja["発表日"].apply(lambda date: pd.to_datetime(date, unit="D", origin=pd.Timestamp("1899/12/30")))

In [0]:
sr = df_kanja["発表日"].value_counts()

In [0]:
# indexに更新日付が含まれるか確認
if dt_update not in sr.index:
    sr[dt_update] = 0

In [0]:
df_patients_sum = sr.sort_index().asfreq("D", fill_value=0).reset_index()

In [0]:
df_patients_sum["日付"] = df_patients_sum["index"].dt.strftime("%Y-%m-%d")

In [0]:
df_patients_sum.rename(columns={"発表日": "小計"}, inplace=True)
df_patients_sum.drop(columns=["index"], inplace=True)

In [249]:
df_patients_sum

,小計,日付
0,1,2020-01-26
1,0,2020-01-27
2,1,2020-01-28
3,0,2020-01-29
4,0,2020-01-30
...,...,...
83,17,2020-04-18
84,10,2020-04-19
85,5,2020-04-20
86,16,2020-04-21


In [0]:
data["patients_summary"] = {
    "data": df_patients_sum.to_dict(orient="records"),
    "date": dt_update.strftime("%Y/%m/%d"),
}

In [0]:
with open("data.json", "w", encoding="utf-8") as fw:
    json.dump(data, fw, ensure_ascii=False, indent=4)